In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/Colab Notebooks/ジャンクション/顔検出/darknet/dataset/WIDER_train_20/face

/content/drive/My Drive/Colab Notebooks/ジャンクション/顔検出/darknet/dataset


In [34]:
ls

wider_face_split/  WIDER_train_20/  WIDER_train_20.zip  WIDER_val/


*画像データファイルの展開

In [0]:
#!unzip WIDER_train_20.zip

#アノテーション作成
YOLO用のAnnotationテキストファイルを各画像ごとに作成<br>
WIDER FACEデータセットのtrainデータから0--Parade〜20--Family_Group（4152枚）を抽出<br>
参考url=https://qiita.com/ha9kberry/items/1bc113cfcd9892a9ddbd

**ディレクトリ構成**<br>
face/<br>
  ┣ 0--Parade/<br>
  ┣ 1--Handshaking/<br>
  ...<br>
  ┣ 20--Family_Group/<br>
  ┣  wider_face_train_bbx_gt.txt<br>
  ┣ generator.ipynb<br>
  ┣ test.txt<br>
  ┗ train.txt<br>

In [58]:
import os
import cv2
import math
import random

# 予め0--Parade〜20--Family_Groupのみ抽出
txt_file = 'wider_face_train_bbx_gt.txt'
with open(txt_file) as f:
    num = f.read().count('jpg')
print('number of images:',num)

number of images: 4152


**Annotationファイル構成**
- 各バウンディングボックスごとにクラス x座標 y座標 ボックス幅 ボックス高さを記載<br>
- 座標はバウンディングボックスの中心座標<br>
- 座標、幅・高さともに、画像の幅・高さに対する相対値<br>
- Annotationファイルは、画像ファイルと同じディレクトリ内に保存<br>
実行時間=3〜5分くらい

In [0]:
#Annotationファイル作成関数
def generator():
  with open(txt_file) as f:
    img_paths=[]
    
    for i in range(num):
      img_path=f.readline().strip()#両端の空白、改行を除去して1行ずつ読み込み
      img_paths.append(img_path)#画像パス一覧に追加
      #cv2を使い画像の読み込み
      im=cv2.imread(img_path)
      im_h=im.shape[0]#高さ
      im_w=im.shape[1]#幅
      split=img_path.split('/')#'/'で画像パスを区切る
      dir_name=split[0]#Annotationファイル格納するディレクトリ名取得
      file_name=split[1].replace('.jpg','.txt')#Annotetionファイル名作成（jpgファイル名をテキストファイル名に変換）
      #txtファイルに書かれている、各画像内のボックス（オブジェクト）の数取得
      count=int(f.readline())
      readline=[]
      readlines=[]
      
      for j in range(count):
        readline=f.readline().split()
        #ボックスの左上座標の取得
        xmin=int(readline[0])
        ymin=int(readline[1])
        #ボックスの幅、高さの取得
        w=int(readline[2])
        h=int(readline[3])
        #ボックスの中央座標（絶対値）の取得
        xcenter=str((xmin+w/2)/im_w)
        ycenter=str((ymin+h/2)/im_h)
        # ボックスの幅・高さを相対値に変換
        w=str(w/im_w)
        h=str(h/im_h)
        
        class_num='0'
        #クラス、x座標、y座標、ボックス幅、ボックス高さを半角スペースで文字列結合
        string=' '.join([class_num, xcenter, ycenter, w, h])
        readlines.append(string)
      #改行で結合
      readlines_str='\n'.join(readlines)
      #該当するディレクトリ内にAnnotationファイルの保存
      with open(dir_name+'/'+file_name,'w') as j:
        j.write(readlines_str)
        
  #画像パス一覧を出力
  return img_paths
      
        

In [0]:
#上記、Annotationファイル作成関数の実行
img_paths=generator()

#訓練データとテストデータを分割する、画像のパス一覧ファイル作成

In [46]:
test_size=0.1
test_num=math.floor(num*test_size)
train_num=num-test_num
print("訓練データ件数：",train_num)
print("テストデータ件数：",test_num)

訓練データ件数： 3737
テストデータ件数： 415


In [0]:
#画像データ（0~20のディレクトリ）が格納されているディレクトリのパス
dir_path="face/"
img_paths=[dir_path+i for i in img_paths]

# trainファイル作成
train_list=img_paths[:train_num]
train_str='\n'.join(train_list)
with open('train.txt', 'w') as f:
    f.write(train_str)

# testファイル作成
test_list=img_paths[train_num:]
test_str='\n'.join(test_list)
with open('test.txt', 'w') as f:
    f.write(test_str)

In [49]:
ls

0--Parade/            17--Ceremony/      5--Car_Accident/
10--People_Marching/  18--Concerts/      6--Funeral/
11--Meeting/          19--Couple/        7--Cheering/
12--Group/            1--Handshaking/    8--Election_Campain/
13--Interview/        20--Family_Group/  9--Press_Conference/
14--Traffic/          2--Demonstration/  test.txt
15--Stock_Market/     3--Riot/           train.txt
16--Award_Ceremony/   4--Dancing/        wider_face_train_bbx_gt.txt
